### PART 3 - iPhone Sentiment Analysis
Note: For this iphone analysis scenario, we have used .filter() method instead of .sample() in Twitter Stream API (in Stream Listener side) as shown below.

<span style="color:blue">l = KafkaListener(tweets_limit=15000) <br>
auth = OAuthHandler(consumer_key, consumer_secret) <br>
auth.set_access_token(access_token, access_secret) <br>
stream = Stream(auth, l)<br>
stream**.filter(track=['iphone'])**</span>

In [1]:
import sys # used to exit
import json, re
import datetime, time

from __future__ import print_function
from textblob import TextBlob

In [2]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils

In [10]:
sc = SparkContext(appName="PythonSparkStreamingKafka")

In [11]:
sc.setLogLevel("WARN")
ssc = StreamingContext(sc,10)
kafkaStream = KafkaUtils.createStream(ssc, 'ip-20-0-32-4.ap-south-1.compute.internal:2181', 'spark-streaming', {'rtd2analysis':1})

In [12]:
ssc.checkpoint("checkpoint")

In [13]:
def cleanTweet(tweet):
    return ' '.join(re.sub("(@[0-9*#+/]+)|([^0-9A-Za-z@:/._ \t])", " ", tweet).split())

In [14]:
def sentimentAnalysis(tweet):
    analysis = TextBlob(tweet)
    snt = analysis.sentiment
    
    if snt.polarity > 0:
        return "positve"
    elif snt.polarity < 0:
        return "negative"
    else:
        return "neutral"
    return analysis.sentiment

In [15]:
parsed = kafkaStream.map(lambda v: json.loads(v[1].encode("ascii", "ignore")))

# Get text part if the tweet 
texts = parsed.map(lambda tweet: cleanTweet(tweet['text']))
sentiments = texts.map(lambda text: (text, sentimentAnalysis(text)))
sentiments.pprint()

In [16]:
try:
    ssc.start()
    ssc.awaitTerminationOrTimeout(1000)
    ssc.stop(stopGraceFully=True,stopSparkContext=True)
    
except KeyboardInterrupt:
    ssc.stop(stopGraceFully=True,stopSparkContext=True)


-------------------------------------------
Time: 2018-08-26 02:39:40
-------------------------------------------
(u'RT @pikachuJihoon: Hey guys. I do iphone 6s plus giveaway because my mom bought iphone 7 plus for me for my birthday. The iphone is not tha', 'neutral')
(u'RT @AgoMarote: ME ROBARON IPHONE 7 PLUS ROSADO EN SANTA FE POR FAVOR DIFUNDIR cualquier informacion puede ser de ayuda GRACIAS', 'neutral')
(u'@askmenfess jangan iphone susah buat download2 illegal wkwk', 'negative')
(u'iPhone https://t.co/rVTGs8TIDU', 'neutral')
(u'iPhone / https://t.co/fgxtFNLTyj', 'neutral')
(u'RT @sentimentos_ai: Tirei 9 na prova Ah eu tirei 10 Eu ralei meu bra o todo Eu j quebrei o bra o Eu ganhei um iPhone 5 Eu j', 'neutral')
(u'iPhone / https://t.co/HwoG8ERXWT', 'neutral')
(u'RT @kyuju32810: what super junior sunbaenims who has been in the music industry for 13 years do behind the scenes expectations: hold i', 'negative')
(u'RT @mopage19: Before amp After // iPhone Edit https://t.co/GjYpGQrbKZ'

-------------------------------------------
Time: 2018-08-26 02:41:20
-------------------------------------------
(u'My best shot at the Moon with this iPhone. yesFilter @ Lake Michigan Chicago Lakefront https://t.co/TiOxHMCSu5', 'positve')
(u'RT @teddyswajjur: I wish iPhone let us unsend messages', 'neutral')
(u'I liked a @YouTube video https://t.co/Fogzi5hKrp D3STROYING CARMEN iPHONE PRANK SHE SNAPPED IN SPANISH', 'positve')
(u'RT @pikachuJihoon: Hey guys. I do iphone 6s plus giveaway because my mom bought iphone 7 plus for me for my birthday. The iphone is not tha', 'neutral')
(u'OMG Check out @kfghsusbeh s video musical.ly gt https://t.co/8LTfRrhfeh', 'neutral')
(u'@CheriseMoniqueF Yeah exactly. I mean I switched from android to iPhone and loved it so it will probably be the sam https://t.co/7WlScZw1Bc', 'positve')
(u'RT dogspetcare: When you put your phone down and the puppies take over apple @CatalystCase waterproof smarties iphone catalystc', 'negative')
(u'FREE iPhone X GIVEAWA